In [ ]:
import pickle
import os
md_path = '/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Third_2019Mar22/'
spotcalls = pickle.load(open(os.path.join(md_path,'results','spotcalls.pkl'),'rb'))

In [ ]:
spotcalls

In [ ]:
from fish_helpers import *

In [ ]:
wounded_spotcalls = spot_dist(wounded_spotcalls)
unwounded_spotcalls = spot_dist(unwounded_spotcalls)
pickle.dump(wounded_spotcalls,open(os.path.join(md_path,'results','wounded_spotcalls.pkl'),'wb'))
pickle.dump(unwounded_spotcalls,open(os.path.join(md_path,'results','unwounded_spotcalls.pkl'),'wb'))

In [ ]:
nucstain = 'nuc_highWellCapacity_6'

In [ ]:
md = Metadata(md_path)
nucstain = 'nuc_highWellCapacity_6'
wounded_intensity_list,wounded_dist_list = nuc_norm_list(wounded_spotcalls,md,nucstain=nucstain)
unwounded_intensity_list,unwounded_dist_list = nuc_norm_list(unwounded_spotcalls,md,nucstain=nucstain)
pickle.dump(wounded_intensity_list,open(os.path.join(md_path,'results','wounded_intensity_list.pkl'),'wb'))
pickle.dump(wounded_dist_list,open(os.path.join(md_path,'results','wounded_dist_list.pkl'),'wb'))
pickle.dump(unwounded_intensity_list,open(os.path.join(md_path,'results','unwounded_intensity_list.pkl'),'wb'))
pickle.dump(unwounded_dist_list,open(os.path.join(md_path,'results','unwounded_dist_list.pkl'),'wb'))

In [ ]:
wounded_intensity_hist,wounded_intensity_bins = nuc_hist(wounded_intensity_list,wounded_dist_list)
unwounded_intensity_hist,unwounded_intensity_bins = nuc_hist(unwounded_intensity_list,unwounded_dist_list)
pickle.dump(wounded_intensity_hist,open(os.path.join(md_path,'results','wounded_intensity_hist.pkl'),'wb'))
pickle.dump(wounded_intensity_bins,open(os.path.join(md_path,'results','wounded_intensity_bins.pkl'),'wb'))
pickle.dump(unwounded_intensity_hist,open(os.path.join(md_path,'results','unwounded_intensity_hist.pkl'),'wb'))
pickle.dump(unwounded_intensity_bins,open(os.path.join(md_path,'results','unwounded_intensity_bins.pkl'),'wb'))

In [ ]:
wounded_gene_df = generate_spatial_df(wounded_spotcalls[wounded_spotcalls.ave>10**2.9],wounded_intensity_bins)
unwounded_gene_df = generate_spatial_df(unwounded_spotcalls[unwounded_spotcalls.ave>10**2.9],unwounded_intensity_bins)
pickle.dump(wounded_gene_df,open(os.path.join(md_path,'results','wounded_gene_df.pkl'),'wb'))
pickle.dump(unwounded_gene_df,open(os.path.join(md_path,'results','unwounded_gene_df.pkl'),'wb'))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(spotcalls.CoordX,bins=1000)
plt.show()

In [ ]:
pickle.dump(spotcalls,open(os.path.join(md_path,'results','spotcalls.pkl'),'wb'))

In [ ]:
from metadata import Metadata
md = Metadata(md_path)

In [ ]:
import numpy as np
xy = np.stack(md.image_table.XY,axis=0)
import matplotlib.pyplot as plt
plt.scatter(xy[:,0],xy[:,1])
plt.show()

In [ ]:
from math import sqrt
import numpy as np
from skimage.transform import resize
stitched = np.empty((5000,11000))
pos_df = {}
for pos in spotcalls.posname.unique():
    if 'Pos' in pos:
        pos_df[pos] = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        x = pos_df[pos][0]+4000
        y = pos_df[pos][1]+1000
        im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0]
        im_resize = resize(im,(int(2048*0.109),int(2048*0.109)),mode='constant')
        x_shape,y_shape=im_resize.shape
        stitched[int(y):int(y+y_shape),int(x):int(x+x_shape)]=im_resize
        upper = np.percentile(stitched[stitched>0],95)
up_stitched = stitched
up_stitched[up_stitched>upper]=upper
plt.imshow(up_stitched,cmap='inferno')

In [ ]:
upper = np.percentile(stitched[stitched>0],95)
up_stitched = stitched
up_stitched[up_stitched>upper]=upper
plt.imshow(up_stitched,cmap='inferno')

In [ ]:
def wavelettransform(stk,wavelet,keep_list='All'):
    if len(stk.shape)==2:
        img = stk
        coeffs = pywt.wavedec2(img,wavelet)
        for i in range(0,len(coeffs)+1):
            if i in keep_list:
                continue
            coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
        ref = pywt.waverec2(coeffs,wavelet)
    else:
        Result = []
        for z in range(stk.shape[2]):
            img = stk[:,:,z]
            coeffs = pywt.wavedec2(img,wavelet)
            for i in range(0,len(coeffs)+1):
                if i in keep_list:
                    continue
                coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
            result = pywt.waverec2(coeffs,wavelet)
            Result.append(result)
        Result  = np.array(Result)
        Result = np.swapaxes(np.swapaxes(Result,0,2),0,1)
        ref = Result.astype('float64')
        ref_mmin = ref-np.percentile(ref.ravel(),1)
        ref = (ref_mmin)/(np.percentile(ref_mmin.ravel(),99))
    return ref

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
#from segmentation_3D import *
import numpy as np
fig = plt.figure(figsize=(8,8))#,dpi=100)
ax = fig.add_subplot(111)
artists = []
X = []
Y = []
scale = int(2048*0.109)
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        X.append(x)
        Y.append(y)
        if x<1000:
            im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0])
            #im_wave = wavelettransform(im,'db9',keep_list=[4,5,6])
            #im_resize = resize(im,(scale,scale),mode='constant',anti_aliasing=True)
            bb = Bbox.from_bounds(x,y,scale,scale)
            bb2 = TransformedBbox(bb,ax.transData)
            bbox_image = BboxImage(bb2,
                                norm = None,
                                origin=None,
                                clip_on=False,
                                cmap='Greys')
            bbox_image.set_data(im)
            ax.add_artist(bbox_image)
ax.set_xlim(-3500,500)
ax.set_ylim(0,3500)
plt.xlabel('X Stage Coordinate')
plt.ylabel('Y Stage Coordinate')
plt.suptitle('Wounded Cornea')
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea')

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
#from segmentation_3D import *
import numpy as np
fig = plt.figure(figsize=(8,8),dpi=1000)
ax = fig.add_subplot(111)
artists = []
X = []
Y = []
scale = int(2048*0.109)
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        X.append(x)
        Y.append(y)
        if x<1000:
            im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0])
            #im_wave = wavelettransform(im,'db9',keep_list=[4,5,6])
            #im_resize = resize(im,(scale,scale),mode='constant',anti_aliasing=True)
            bb = Bbox.from_bounds(x,y,scale,scale)
            bb2 = TransformedBbox(bb,ax.transData)
            bbox_image = BboxImage(bb2,
                                norm = None,
                                origin=None,
                                clip_on=False,
                                cmap='Greys')
            bbox_image.set_data(im)
            ax.add_artist(bbox_image)
gene = 'Tnfrsf21'
temp = spotcalls[(spotcalls.gene==gene)&(spotcalls.CoordX<1000)&(spotcalls.ave>1000)]
x = temp.CoordX+2049*0.109
y = temp.CoordY
z = temp.z
plt.scatter(x,y,s=1,alpha=0.1,c='r')
ax.set_xlim(-3500,500)
ax.set_ylim(0,3500)
plt.xlabel('X Stage Coordinate')
plt.ylabel('Y Stage Coordinate')
plt.suptitle('Wounded Cornea')
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea_spots')

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
#from segmentation_3D import *
import numpy as np
fig = plt.figure(figsize=(8,8),dpi=1000)
ax = fig.add_subplot(111)
artists = []
x_min = -2500
x_max = -1300
y_min = 600
y_max = 2000
scale = int(2048*0.109)
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        if (x>x_min)&(x<x_max)&(y>y_min)&(y<y_max):
            im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0])
            #im_wave = wavelettransform(im,'db9',keep_list=[4,5,6])
            #im_resize = resize(im,(scale,scale),mode='constant',anti_aliasing=True)
            bb = Bbox.from_bounds(x,y,scale,scale)
            bb2 = TransformedBbox(bb,ax.transData)
            bbox_image = BboxImage(bb2,
                                norm = None,
                                origin=None,
                                clip_on=False,
                                cmap='Greys')
            bbox_image.set_data(im)
            ax.add_artist(bbox_image)
gene = 'Tnfrsf21'
temp = spotcalls[(spotcalls.gene==gene)&(spotcalls.CoordX<1000)&(spotcalls.ave>1000)]
x = temp.CoordX+2049*0.109
y = temp.CoordY
z = temp.z
plt.scatter(x,y,s=1,alpha=0.1,c='r')
ax.set_xlim(x_min,x_max)
ax.set_ylim(y_min,y_max)
plt.xlabel('X Stage Coordinate')
plt.ylabel('Y Stage Coordinate')
plt.suptitle('Wounded Cornea')
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea_spots_zoom')

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline
gene = 'Tnfrsf21'
temp = spotcalls[(spotcalls.gene==gene)&(spotcalls.CoordX<1000)]
x = temp.CoordX+2049*0.109
y = temp.CoordY
z = temp.z
plt.scatter(x,y,s=1,alpha=0.1)
plt.show()

In [ ]:
for gene in spotcalls.gene.unique():
    temp = spotcalls[(spotcalls.gene==gene)&(spotcalls.CoordX<1000)]
    x = temp.CoordX+2049*0.109
    y = temp.CoordY
    z = temp.z
    plt.scatter(x,y,s=1,alpha=0.1)
    plt.title(gene)
    plt.show()

In [ ]:
gene = 'Tnfrsf21'
temp = spotcalls[(spotcalls.gene==gene)&(spotcalls.CoordX<1000)&(spotcalls.ave>1000)]
x = temp.CoordX+2049*0.109
y = temp.CoordY
z = temp.z
plt.scatter(x,y,c=z,s=1)
plt.show()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
#from segmentation_3D import *
import numpy as np
fig = plt.figure(figsize=(8,8),dpi=1000)
ax = fig.add_subplot(111)
artists = []
X = []
Y = []
scale = int(2048*0.109)
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        X.append(x)
        Y.append(y)
        if x<1000:
            im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0])
            #im_wave = wavelettransform(im,'db9',keep_list=[4,5,6])
            #im_resize = resize(im,(scale,scale),mode='constant',anti_aliasing=True)
            bb = Bbox.from_bounds(x,y,scale,scale)
            bb2 = TransformedBbox(bb,ax.transData)
            bbox_image = BboxImage(bb2,
                                norm = None,
                                origin=None,
                                clip_on=False,
                                cmap='Greys')
            bbox_image.set_data(im)
            ax.add_artist(bbox_image)
ax.set_xlim(-3500,500)
ax.set_ylim(0,3500)
plt.xlabel('X Stage Coordinate')
plt.ylabel('Y Stage Coordinate')
plt.suptitle('Wounded Cornea')
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea')

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
import numpy as np
fig = plt.figure(figsize=(8,8),dpi=100)
ax = fig.add_subplot(111)
artists = []
X = []
Y = []
scale = int(2048*0.109)
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        X.append(x)
        Y.append(y)
        if x>1000:
            im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0])
            im_resize = resize(im,(scale,scale),mode='constant',anti_aliasing=True)
            bb = Bbox.from_bounds(x,y,scale,scale)
            bb2 = TransformedBbox(bb,ax.transData)
            bbox_image = BboxImage(bb2,
                                norm = None,
                                origin=None,
                                clip_on=False,
                                cmap='Greys')
            bbox_image.set_data(im_resize)
            ax.add_artist(bbox_image)
ax.set_xlim(3000,7000)
ax.set_ylim(-500,3000)
plt.xlabel('X Stage Coordinate')
plt.ylabel('Y Stage Coordinate')
#plt.suptitle('mRNA Spots Identified Single Cornea')
plt.show()
fig.savefig('/home/zach/Desktop/Stitched_unwounded')

In [ ]:
wounded_pos = []
unwounded_pos = []
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        if x<1000:
            wounded_pos.append(pos)
        else:
            unwounded_pos.append(pos)

In [ ]:
keepers= []
tossers = []
for pos in spotcalls.posname.unique():
    print(pos)
    if np.average(spotcalls[spotcalls.posname==pos].ave)<1250:
        tossers.append(pos)
        #print(pos,' Tossed')
    else:
        keepers.append(pos)
        #print(pos,' Good')
print('Total Good: ',len(keepers))
print('Total Tossed: ',len(tossers))

In [ ]:
spotcalls_good = spotcalls
for pos in tossers:
    spotcalls_good = spotcalls_good.drop(index=spotcalls_good[spotcalls_good.posname==pos].index)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
temp_df = spotcalls_good
plt.hist(temp_df.ave,bins=int(len(temp_df.ave)/1000))
plt.xlim([0,3500])
plt.suptitle('Distribution of FISH spot intensity')
plt.xlabel('Spot intensity')
plt.ylabel('Spot Count')
plt.show()

In [ ]:
from math import sqrt
import numpy as np
from skimage.transform import resize
stitched = np.empty((5000,11000))
pos_df = {}
for pos in spotcalls_good.posname.unique():
    if 'Pos' in pos:
        pos_df[pos] = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        x = pos_df[pos][0]+4000
        y = pos_df[pos][1]+1000
        im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0]
        im_resize = resize(im,(int(2048*0.109),int(2048*0.109)),mode='constant')
        x_shape,y_shape=im_resize.shape
        stitched[int(y):int(y+y_shape),int(x):int(x+x_shape)]=im_resize
upper = np.percentile(stitched[stitched>0],95)
up_stitched = stitched
up_stitched[up_stitched>upper]=upper
plt.imshow(up_stitched,cmap='inferno')

In [ ]:
center_unwounded = [4750,1750]
center_wounded = [-1750,1500]

In [ ]:
from math import sqrt
import numpy as np
from skimage.transform import resize

center_unwounded = [4750,1750]
center_wounded = [-1750,1500]
wounded_distlist = []
wounded_nuclist = []
unwounded_distlist = []
unwounded_nuclist = []
stitched = np.empty((4000,7500))
temp = spotcalls_filtered
for pos in temp.posname.unique():
    x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
    im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0]
    im_resize = resize(im,(int(2048*0.109),int(2048*0.109)),mode='constant',anti_aliasing=True)
    if x<1000:
        for x_loc in range(im_resize.shape[1]):
            x_vector = im_resize[:,x_loc]
            for y_loc,pixel in enumerate(x_vector):
                pix_x = x+x_loc
                pix_y = y+y_loc
                distance = sqrt( (center_wounded[0] - pix_x)**2 + (center_wounded[1] - pix_y)**2 )
                wounded_distlist.append(distance)
                wounded_nuclist.append(pixel)
    if x>1000:
        for x_loc in range(im_resize.shape[1]):
            x_vector = im_resize[:,x_loc]
            for y_loc,pixel in enumerate(x_vector):
                if pixel==0:
                    continue
                pix_x = x+x_loc
                pix_y = y+y_loc
                distance = sqrt( (center_unwounded[0] - pix_x)**2 + (center_unwounded[1] - pix_y)**2 )
                unwounded_distlist.append(distance)
                unwounded_nuclist.append(pixel)

In [ ]:
wounded_nuc_intensity_dict=pickle.load(open(os.path.join(md_path,'results','wounded_nuc_intensity_dict.pkl'),'rb'))
unwounded_nuc_intensity_dict=pickle.load(open(os.path.join(md_path,'results','unwounded_nuc_intensity_dict.pkl'),'rb'))

In [ ]:
nbins = 100
rmin = 150
rmax = 1600
bins = np.linspace(rmin,rmax,nbins)
bins[1]-bins[0]

In [ ]:
nbins = 100
rmin = 150
rmax = 1600
bins = np.linspace(rmin,rmax,nbins)
step = (bins[1]-bins[0])/2
wounded_nuc_intensity_dict = np.zeros_like(bins)
unwounded_nuc_intensity_dict = np.zeros_like(bins)
wounded_nuclist_array = np.array(wounded_nuclist)
unwounded_nuclist_array = np.array(unwounded_nuclist)
wounded_distlist_array = np.array(wounded_distlist)
unwounded_distlist_array = np.array(unwounded_distlist)
for i,r in enumerate(bins):
    r_min = r-step
    r_max = r+step
    wounded_keepers = np.array(wounded_nuclist)
    unwounded_keepers = np.array(unwounded_nuclist)
    wounded_keepers[wounded_distlist_array>r_max]=0
    unwounded_keepers[unwounded_distlist_array>r_max]=0
    wounded_keepers[wounded_distlist_array<r_min]=0
    unwounded_keepers[unwounded_distlist_array<r_min]=0
    wounded_nuc_intensity_dict[i]=np.sum(wounded_keepers)
    unwounded_nuc_intensity_dict[i]=np.sum(unwounded_keepers)

In [ ]:
import numpy as np
nbins = 100
rmin = 150
rmax = 1600
bins = np.linspace(rmin,rmax,nbins)
wounded_nuc_intensity_dict = pickle.load(open(os.path.join(md_path,'results','wounded_nuc_intensity_dict.pkl'),'rb'))
unwounded_nuc_intensity_dict = pickle.load(open(os.path.join(md_path,'results','unwounded_nuc_intensity_dict.pkl'),'rb'))

In [ ]:
wounded_nuc_intensity = wounded_nuc_intensity_dict
unwounded_nuc_intensity = unwounded_nuc_intensity_dict

In [ ]:
plt.plot(bins[1:-1],wounded_nuc_intensity_dict[1:-1],c='r',label='wounded')
plt.plot(bins[1:-1],unwounded_nuc_intensity_dict[1:-1],c='k',label='unwounded')
plt.legend()
plt.xlabel('distance from center (um)')
plt.ylabel('nuclear intensity')
plt.legend()
plt.show()

In [ ]:
plt.plot(bins[1:-1],wounded_nuc_intensity_dict[1:-1],c='r',label='wounded')
plt.plot(bins[1:-1],unwounded_nuc_intensity_dict[1:-1],c='k',label='unwounded')
plt.legend()
plt.xlabel('distance from center (um)')
plt.ylabel('nuclear intensity')
plt.legend()
plt.show()

In [ ]:
from metadata import Metadata
import pickle
import os
X = []
Y = []
pixelsize = 0.109
cameradirection = [1,1]
temp = spotcalls
for pos in temp.posname.unique():
    CoordXY = md.image_table[md.image_table.Position==pos].XY.iloc[0]
    coordX = CoordXY[0]
    coordY = CoordXY[1]
    pos_temp = temp[temp.posname==pos]
    pos_centroid = pos_temp.centroid
    for xy in pos_centroid:
        rx = xy[1]-2048
        ry = xy[0]#-2048
        x = rx*pixelsize*cameradirection[0]+coordX
        y = ry*pixelsize*cameradirection[1]+coordY
        X.append(x)
        Y.append(y)
spotcalls['CoordX'] = X
spotcalls['CoordY'] = Y
from math import sqrt
center_unwounded = [4750,1750]
center_wounded = [-1750,1500]
distance_list = []
for i in range(len(spotcalls)):
    if (i % 100000)==0:
        print(i)
    X = spotcalls.CoordX.iloc[i]
    Y = spotcalls.CoordY.iloc[i]
    if X<1000:
        distance = sqrt( (center_wounded[0] - X)**2 + (center_wounded[1] - Y)**2 )
    else:
        distance = sqrt( (center_unwounded[0] - X)**2 + (center_unwounded[1] - Y)**2 )
    distance_list.append(distance)
spotcalls['distance']=distance_list
pickle.dump(spotcalls,open(os.path.join(md_path,'results','spotcalls.pkl'),'wb'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
%matplotlib inline
hist_bins = np.zeros(len(bins)+1)
bin_dist = (bins[1]-bins[0])/2
hist_bins[:-1] = bins-bin_dist
hist_bins[-1] = bins[99]+bin_dist
hist_bins
temp_spotcalls = spotcalls[spotcalls.ave>1000]
plt.hist(temp_spotcalls[temp_spotcalls.CoordX<1000].distance,bins=hist_bins,color='r',alpha=0.5,label='Wounded')
plt.hist(temp_spotcalls[temp_spotcalls.CoordX>1000].distance,bins=hist_bins,color='k',alpha=0.5,label='Unwounded')
plt.legend()
plt.xlabel('distance from center (um)')
plt.ylabel('Spot Abundance')
plt.legend()
plt.show()

min_bin = 7
nuc_wounded = wounded_nuc_intensity_dict
nuc_unwounded = unwounded_nuc_intensity_dict

plt.plot(bins[min_bin:-1],nuc_wounded[min_bin:-1],c='r',label='wounded')
plt.plot(bins[min_bin:-1],nuc_unwounded[min_bin:-1],c='k',label='unwounded')
plt.legend()
plt.xlabel('distance from center (um)')
plt.ylabel('nuclear intensity')
plt.legend()
plt.show()

unwounded_dist_hist = {}
wounded_dist_hist = {}
for gene in temp_spotcalls.gene.unique():  
    temp_df = temp_spotcalls[(temp_spotcalls.CoordX<1000)&(temp_spotcalls.gene==gene)]
    temp = plt.hist(temp_df.distance,bins=hist_bins,color='r',alpha=0.5,label='Wounded')
    temp = np.nan_to_num(temp[0])
    temp_norm = temp/nuc_wounded
    wounded_dist_hist[gene] = temp_norm
    
    temp_df = temp_spotcalls[(temp_spotcalls.CoordX>1000)&(temp_spotcalls.gene==gene)]
    temp = plt.hist(temp_df.distance,bins=hist_bins,color='k',alpha=0.5,label='Unwounded')
    temp = np.nan_to_num(temp[0])
    temp_norm = temp/nuc_unwounded
    unwounded_dist_hist[gene] = temp_norm
    
    plt.title(gene+' Raw')
    plt.xlabel('distance from center unwounded (um)')
    plt.ylabel('spots/nuclear intensity')
    plt.legend()
    plt.show()

    plt.plot(bins[min_bin:-1],unwounded_dist_hist[gene][min_bin:-1],color='k',label='Unwounded')
    plt.plot(bins[min_bin:-1],wounded_dist_hist[gene][min_bin:-1],color='r',label='Wounded')
    plt.title(gene+' Normalized by nuclear intensity')
    plt.xlabel('distance from center(um)')
    plt.ylabel('spots/nuclear intensity')
    plt.legend()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
%matplotlib inline
hist_bins = np.zeros(len(bins)+1)
bin_dist = (bins[1]-bins[0])/2
hist_bins[:-1] = bins-bin_dist
hist_bins[-1] = bins[99]+bin_dist
hist_bins
temp_spotcalls = spotcalls_filtered
plt.hist(temp_spotcalls[temp_spotcalls.CoordX<1000].distance,bins=hist_bins,color='r',alpha=0.5,label='Wounded')
plt.hist(temp_spotcalls[temp_spotcalls.CoordX>1000].distance,bins=hist_bins,color='k',alpha=0.5,label='Unwounded')
plt.legend()
plt.xlabel('distance from center (um)')
plt.ylabel('Spot Abundance')
plt.legend()
plt.show()

min_bin = 7
nuc_wounded = wounded_nuc_intensity_dict
nuc_unwounded = unwounded_nuc_intensity_dict

plt.plot(bins[min_bin:-1],nuc_wounded[min_bin:-1],c='r',label='wounded')
plt.plot(bins[min_bin:-1],nuc_unwounded[min_bin:-1],c='k',label='unwounded')
plt.legend()
plt.xlabel('distance from center (um)')
plt.ylabel('nuclear intensity')
plt.legend()
plt.show()

unwounded_dist_hist = {}
wounded_dist_hist = {}
for gene in temp_spotcalls.gene.unique():  
    temp_df = temp_spotcalls[(temp_spotcalls.CoordX<1000)&(temp_spotcalls.gene==gene)]
    temp = plt.hist(temp_df.distance,bins=hist_bins,color='r',alpha=0.5,label='Wounded')
    temp = np.nan_to_num(temp[0])
    temp_norm = temp/nuc_wounded
    wounded_dist_hist[gene] = temp_norm
    
    temp_df = temp_spotcalls[(temp_spotcalls.CoordX>1000)&(temp_spotcalls.gene==gene)]
    temp = plt.hist(temp_df.distance,bins=hist_bins,color='k',alpha=0.5,label='Unwounded')
    temp = np.nan_to_num(temp[0])
    temp_norm = temp/nuc_unwounded
    unwounded_dist_hist[gene] = temp_norm
    
    plt.title(gene+' Raw')
    plt.xlabel('distance from center unwounded (um)')
    plt.ylabel('spots/nuclear intensity')
    plt.legend()
    plt.show()

    plt.plot(bins[min_bin:-1],unwounded_dist_hist[gene][min_bin:-1],color='k',label='Unwounded')
    plt.plot(bins[min_bin:-1],wounded_dist_hist[gene][min_bin:-1],color='r',label='Wounded')
    plt.title(gene+' Normalized by nuclear intensity')
    plt.xlabel('distance from center(um)')
    plt.ylabel('spots/nuclear intensity')
    plt.legend()
    plt.show()

In [ ]:
%matplotlib inline 
from scipy.stats import gaussian_kde
plt.figure(figsize=(8,8))
plt.imshow(up_stitched,cmap='Greys')
gene = 'Tnfrsf21'
temp = spotcalls[spotcalls.gene==gene]
x = temp.CoordX+4000+int(2048*0.109)
y = temp.CoordY+1000
z = temp.z
plt.scatter(x,y,c=z,s=1)
plt.xlim([1000,3000])
plt.ylim([1500,3500])
plt.show()

In [ ]:
from math import sqrt
import numpy as np
from skimage.transform import resize
stitched_all = np.empty((5000,11000))
pos_df = {}
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        pos_df[pos] = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        x = pos_df[pos][0]+4000
        y = pos_df[pos][1]+1000
        im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0]
        im_resize = resize(im,(int(2048*0.109),int(2048*0.109)),mode='constant',anti_aliasing=True)
        x_shape,y_shape=im_resize.shape
        stitched_all[int(y):int(y+y_shape),int(x):int(x+x_shape)]=im_resize
upper = np.percentile(stitched_all[stitched_all>0],95)
up_stitched_all = stitched_all
up_stitched_all[up_stitched_all>upper]=upper
plt.imshow(up_stitched_all,cmap='inferno')

In [ ]:
X = []
Y = []
for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        X.append(x)
        Y.append(y)
plt.scatter(X,Y)

In [ ]:
plt.scatter(X,Y)

In [ ]:
edge_pos=[]
x_min=1500-4000
x_max=2750-4000
y_min=1000-1000
y_max=2000-1000

for pos in md.image_table.Position.unique():
    x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
    if (x>x_min)&(x<x_max)&(y>y_min)&(y<y_max):
        edge_pos.append(pos)

In [ ]:
file_pos = []
for pos in edge_pos:
    file_pos.append(md.image_table[md.image_table.Position==pos].root_pth.iloc[0].split('/')[0])

In [ ]:
from math import sqrt
import numpy as np
from skimage.transform import resize
stitched_wound = np.empty((1800,1400))
pos_df = {}
for pos in edge_pos:
    if 'Pos' in pos:
        pos_df[pos] = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        x = pos_df[pos][0]+2500
        y = pos_df[pos][1]
        im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0]
        im_resize = resize(im,(int(2048*0.109),int(2048*0.109)),mode='constant',anti_aliasing=True)
        x_shape,y_shape=im_resize.shape
        stitched_wound[int(y):int(y+y_shape),int(x):int(x+x_shape)]=im_resize
upper = np.percentile(stitched_wound[stitched_wound>0],95)
#stitched_wound[stitched_wound>upper]=upper
plt.imshow(stitched_wound,cmap='inferno')

In [ ]:
plt.imshow(up_stitched_all,cmap='inferno')
plt.xlim([500,4000])
plt.ylim([1000,4000])
plt.show()

In [ ]:
pos = 'Pos150'
im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=20)[:,:,0]
plt.imshow(im)
plt.show()

In [ ]:
plt.hist(im.ravel())

In [ ]:
plt.imshow(im>2000)
plt.show()

In [ ]:
from scipy import ndimage
from skimage import morphology,filters

temp = up_stitched_all.copy()
temp = filters.gaussian(temp,sigma=5)
temp[temp>0.02]=0
im_bin = temp>0
im_bin = ndimage.binary_dilation(im_bin,iterations=5)
im_bin = morphology.remove_small_holes(im_bin, 2000)
im_bin = morphology.remove_small_objects(im_bin, 10000)
im_bin = morphology.erosion(im_bin)
plt.imshow(im_bin,cmap='inferno')
plt.xlim([500,4000])
plt.ylim([1000,4000])
plt.show()

In [ ]:
from skimage.measure import label
label_img = label(im_bin)
from skimage.measure import regionprops
regions = regionprops(label_img)


In [ ]:
for region in regions:
    plt.imshow(region.image)
    plt.title(region.area)
    plt.show()

In [ ]:
mask = regions[2]

In [ ]:
plt.imshow(mask.image)

In [ ]:
mask.coords

In [ ]:
plt.imshow(up_stitched_all,cmap='inferno')
plt.imshow()
plt.xlim([500,4000])
plt.ylim([1000,4000])
plt.show()

In [ ]:
%matplotlib inline 
from scipy.stats import gaussian_kde
plt.figure(figsize=(8,8))
plt.imshow(up_stitched,cmap='Greys')
gene = 'Tnfrsf21'
x_offset = 4000+int(2048*0.109)
y_offset = 1000
x_min = 2250
x_max = 3000
y_min = 2400
y_max = 2600
temp = spotcalls[(spotcalls.gene==gene)&(spotcalls.CoordY>(y_min-y_offset))&(spotcalls.CoordY<(y_max-y_offset))&(spotcalls.CoordX>(x_min-x_offset))&(spotcalls.CoordX<(x_max-x_offset))]
x = temp.CoordX+x_offset
y = temp.CoordY+y_offset
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)
plt.scatter(x,y,c=z,s=1)
plt.xlim([x_min,x_max])
plt.ylim([y_min,y_max])
plt.show()

In [ ]:
%matplotlib inline 
from scipy.stats import gaussian_kde
x_offset = 4000+int(2048*0.109)
y_offset = 1000
x_min = 2250
x_max = 3000
y_min = 2400
y_max = 2600
temp_mask = []
temp_df = spotcalls[(spotcalls.ave>1000)&(spotcalls.CoordY>(y_min-y_offset))&(spotcalls.CoordY<(y_max-y_offset))&(spotcalls.CoordX>(x_min-x_offset))&(spotcalls.CoordX<(x_max-x_offset))]
for gene in spotcalls.gene.unique():
    plt.figure(figsize=(8,8))
    plt.imshow(up_stitched,cmap='Greys')
    temp = spotcalls[(spotcalls.gene==gene)]
    x = temp.CoordX+x_offset
    y = temp.CoordY+y_offset
    #xy = np.vstack([x,y])
    z = temp.z#gaussian_kde(xy)(xy)
    plt.scatter(x,y,c=z,s=1)
    plt.xlim([x_min,x_max])
    plt.ylim([y_min,y_max])
    plt.title(gene)
    plt.show()

In [ ]:
%matplotlib inline 
from matplotlib import pyplot as plt
import matplotlib as mpl
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
from sklearn.preprocessing import normalize
from scipy import ndimage
from skimage import morphology,filters
import pywt
import numpy as np
from scipy.stats import gaussian_kde
x_offset = int(2048*0.109)
y_offset = 0
x_min = -2000
x_max = -1100
y_min = 1300
y_max = 1400
# temp_df = spotcalls[(spotcalls.ave>1000)&
#                     (spotcalls.CoordY>(y_min-y_offset))&
#                     (spotcalls.CoordY<(y_max-y_offset))&
#                     (spotcalls.CoordX>(x_min-x_offset))&
#                     (spotcalls.CoordX<(x_max-x_offset))]
img_dict = {}
img_x_dict = {}
img_y_dict = {}
wavelet = 'db10'
keep_list=[4,5,6]
scale = int(2048*0.109)
for pos in md.image_table.Position.unique():
    x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
    if not (x>x_min)&(x<x_max)&(y>y_min)&(y<y_max):
        continue
    img_x_dict[pos]=x
    img_y_dict[pos]=y
    im = np.flipud(md.stkread(Position=pos,
                              Channel='DeepBlue',
                              acq='nuc_highWellCapacity_6',
                              Zindex=10)[:,:,0])
    thresh = np.percentile(im.ravel(),99)
    im[im>thresh]=thresh
    coeffs = pywt.wavedec2(im,wavelet)
    for i in range(0,len(coeffs)+1):
        if i in keep_list:
            continue
        coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
    Result = pywt.waverec2(coeffs,wavelet)
    thresh = np.percentile(Result.ravel(),99.9)
    Result[Result>thresh]=thresh
    Result[im<2**8.5]=np.nan
    if (x>-2500)&(x<-500)&(y>800)&(y<2000):
        im_bin = im>500
    else:
        im_bin = im>2000
    im_bin = ndimage.binary_dilation(im_bin,iterations=5)
    im_bin = morphology.remove_small_holes(im_bin, 2000)
    im_bin = morphology.remove_small_objects(im_bin, 10000)
    im_bin = morphology.erosion(im_bin)
    Result[im_bin==False]=np.nan
    Result = Result-np.nanpercentile(Result.ravel(),1)
    Result = Result/np.nanpercentile(Result.ravel(),99)
    img_dict[pos]=Result

temp_mask = []
poses = img_dict.keys()
for pos in spotcalls.posname:
    if pos in poses:
        temp_mask.append(True)
    else:
        temp_mask.append(False)
temp_df = spotcalls[(spotcalls.ave>1000)&temp_mask]


In [ ]:
norm = mpl.colors.Normalize(vmin=0,vmax=1)
BIGGER_SIZE = 12
plt.rc('font', size=BIGGER_SIZE, family='Arial') # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

temp_df = spotcalls[(spotcalls.ave>1000)&temp_mask]
for gene in temp_df.gene.unique():
    X = []
    Y = []
    temp = temp_df[(temp_df.gene==gene)]
    if len(temp)<100:
        continue
    fig = plt.figure(figsize=(4,1))
    ax = fig.add_subplot(111)
    for pos in img_dict.keys():
        x = img_x_dict[pos]
        X.append(x)
        y = img_y_dict[pos]
        Y.append(Y)
        bb = Bbox.from_bounds(x,y,scale,scale)
        bb2 = TransformedBbox(bb,ax.transData)
        bbox_image = BboxImage(bb2,
                            norm = norm,
                            origin=None,
                            clip_on=False,
                            cmap='Greys')
        bbox_image.set_data(img_dict[pos])
        ax.add_artist(bbox_image) 
    
    x = temp.CoordX+x_offset
    y = temp.CoordY+y_offset
    z = temp.z
    plt.scatter(x,y,c=z,s=1)
    plt.xlim([np.min(X),np.max(X)+scale])
    plt.ylim([np.min(Y),np.max(Y)+scale])
    plt.title(gene)
    plt.show()

In [ ]:
pickle.dump(wounded_dist_hist,open(os.path.join(md_path,'results','wounded_dist_hist'),'wb'))
pickle.dump(unwounded_dist_hist,open(os.path.join(md_path,'results','unwounded_dist_hist'),'wb'))

In [ ]:
wounded_dist_hist = pickle.load(open(os.path.join(md_path,'results','wounded_dist_hist'),'rb'))
unwounded_dist_hist = pickle.load(open(os.path.join(md_path,'results','unwounded_dist_hist'),'rb'))

In [ ]:
import pandas as pd
log_fold_change = np.zeros((len(spotcalls.gene.unique()),nbins))
unwounded = np.zeros((len(spotcalls.gene.unique()),nbins))
wounded = np.zeros((len(spotcalls.gene.unique()),nbins))
for i,gene in enumerate(wounded_dist_hist.keys()):
    wounded_gene = wounded_dist_hist[gene]+.00001
    unwounded_gene = unwounded_dist_hist[gene]+.00001
    log = np.log2(wounded_gene/unwounded_gene)
    unwounded[i,:] = np.log2(unwounded_gene)
    wounded[i,:] = np.log2(wounded_gene)
    log_fold_change[i,:] = log

log_fold_change_df = pd.DataFrame(data=log_fold_change,columns=bins,index=wounded_dist_hist.keys())
wounded_df = pd.DataFrame(data=wounded,columns=bins,index=wounded_dist_hist.keys())
unwounded_df = pd.DataFrame(data=unwounded,columns=bins,index=wounded_dist_hist.keys())

In [ ]:
import seaborn as sns
sns.clustermap(log_fold_change_df,col_cluster=False,cmap='bwr',center=0)

In [ ]:
import seaborn as sns
sns.clustermap(log_fold_change_df,col_cluster=False,cmap='bwr',center=0)

In [ ]:
import seaborn as sns
sns.clustermap(log_fold_change_df,col_cluster=False,cmap='bwr',center=0,z_score=True)

In [ ]:
import seaborn as sns
sns.clustermap(wounded_df,col_cluster=False,cmap='bwr',z_score=True,center=0)

In [ ]:
import seaborn as sns
sns.clustermap(unwounded_df,col_cluster=False,cmap='bwr',z_score=True,center=0)

In [ ]:
import pickle
import os
from fish_results import HybeData
import pandas as pd
md_path = '/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Third_2019Mar22/'
cstk_path = os.path.join(md_path,'codestacks')
subset = pickle.load(open(os.path.join(cstk_path,'subset.pkl'),'rb'))
#hybedatas = [HybeData(os.path.join(cstk_path, i)) for i in subset]
spotcalls = []
poses = [i for i in os.listdir(cstk_path) if os.path.isdir(os.path.join(cstk_path, i))]
for pos in poses:
    hdata = HybeData(os.path.join(cstk_path, pos))
    if len(hdata.metadata) == 0:
        print(pos)
        hdata.regenerate_metadata()
    hdata = HybeData(os.path.join(cstk_path, pos))
    if len(hdata.metadata) == 0:
        print(pos)
        self = hdata
        self.metadata = pandas.DataFrame(columns=['posname', 'zindex', 'dtype', 'filename'])
        dtypes = ['nf','cstk','cimg','mask','beads','tforms','spotcalls']
        for relative_fname in os.listdir(self.base_path):
            try:
                dtype = relative_fname.split('_')[0]
                zindex = relative_fname.split('_')[-1].split('.')[0]
                posname = relative_fname.split(dtype+'_')[1].split('_z')[0]
                if dtype in dtypes:
                    self.metadata = self.metadata.append({'posname': posname, 'zindex': str(zindex),
                                                          'dtype': dtype, 'filename': relative_fname},
                                                         ignore_index=True)
            except:
                print(relative_fname)
                continue
        self.metadata = self.metadata.drop_duplicates()
        self.metadata.to_csv(os.path.join(self.base_path, self.file_name),index=False)

In [ ]:
import pickle
import os
cstk_path = os.path.join(md_path,'codestacks')
spotcalls = pickle.load(open(os.path.join(cstk_path,'spotcalls.pkl'),'rb'))

In [ ]:
from analysis_scripts.iterative_classify_codestacks_and_parse import *
poses = [i for i in os.listdir(cstk_path) if os.path.isdir(os.path.join(cstk_path, i))]
finished_pos = find_subset(cstk_path,400,18)
print(len(finished_pos))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from scipy.stats import spearmanr
import os
ReadsPerGene = pd.read_excel('/bigstore/GeneralStorage/Zach/Cornea_RNAseq/Aligned/ReadsPerGene.xlsx')
GeneList = pd.read_excel('/bigstore/GeneralStorage/Zach/MERFISH/Inflammatory/InflammationGeneList.xlsx')
vals = []
FISH_Spots = Counter(spotcalls.gene)
for gn,cc in FISH_Spots.items():
    if 'blank' in gn:
        continue
    else:
        gid = GeneList.Gene_ID[GeneList.Gene==gn].tolist()[0]
        counts = ReadsPerGene[ReadsPerGene.GeneIDs==gid].Unstranded.values
        fpkm = counts/GeneList.Length[GeneList.Gene==gn].tolist()[0]
        if len(fpkm)>0:
            if cc<2:
                continue
            vals.append((fpkm[0],cc))
fpkms,ccs = zip(*vals)
from scipy.stats import spearmanr
plt.scatter(np.log2(fpkms),np.log2(ccs))
print(spearmanr(fpkms,ccs))

In [ ]:
import pandas as pd
log_fold_change = np.zeros((len(spotcalls.gene.unique()),nbins))
unwounded = np.zeros((len(spotcalls.gene.unique()),nbins))
wounded = np.zeros((len(spotcalls.gene.unique()),nbins))
for i,gene in enumerate(wounded_dist_hist.keys()):
    wounded_gene = wounded_dist_hist[gene]
    unwounded_gene = unwounded_dist_hist[gene]
    log = np.log2(wounded_gene/unwounded_gene)
    unwounded[i,:] = unwounded_gene
    wounded[i,:] = wounded_gene
    log_fold_change[i,:] = log

log_fold_change_df = pd.DataFrame(data=log_fold_change,columns=bins,index=wounded_dist_hist.keys())
wounded_df = pd.DataFrame(data=wounded,columns=bins,index=wounded_dist_hist.keys())
unwounded_df = pd.DataFrame(data=unwounded,columns=bins,index=wounded_dist_hist.keys())

In [ ]:
plt.figure(figsize=(12,6),dpi=120)
for i in range(len(wounded_df)):
    rsd = np.std(np.array(wounded_df.iloc[i]))/np.std(np.array(wounded_df.iloc[i]))
    if >1.2:
        plt.plot(bins,np.array(wounded_df.iloc[i]),label=wounded_df.index[i])
plt.rc('font', size=15) 
plt.rc('axes', titlesize=15)
plt.title('Gene Expression Patterns')
plt.legend(loc='right',bbox_to_anchor=(1.2,0.5))
plt.xlabel('Distance From Center of Wound (um)')
plt.ylabel('Log2 Fold Change in Expression')

plt.show()

In [ ]:
import pandas as pd
log_fold_change = np.zeros((len(spotcalls.gene.unique()),nbins))
unwounded = np.zeros((len(spotcalls.gene.unique()),nbins))
wounded = np.zeros((len(spotcalls.gene.unique()),nbins))
for i,gene in enumerate(wounded_dist_hist.keys()):
    wounded_gene = wounded_dist_hist[gene]
    unwounded_gene = unwounded_dist_hist[gene]
    log = np.log2(wounded_gene/unwounded_gene)
    unwounded[i,:] = unwounded_gene
    wounded[i,:] = wounded_gene
    log_fold_change[i,:] = log

log_fold_change_df = pd.DataFrame(data=log_fold_change,columns=bins,index=wounded_dist_hist.keys())
wounded_df = pd.DataFrame(data=wounded,columns=bins,index=wounded_dist_hist.keys())
unwounded_df = pd.DataFrame(data=unwounded,columns=bins,index=wounded_dist_hist.keys())
plt.figure(figsize=(12,6),dpi=120)
for i in range(len(wounded_df)):
    rsd = np.std(np.array(wounded_df.iloc[i]))/np.mean(np.array(wounded_df.iloc[i]))
    if rsd>1.4:
        plt.plot(bins,np.log2(np.array(wounded_df.iloc[i])),label=wounded_df.index[i])
plt.rc('font', size=15) 
plt.rc('axes', titlesize=15)
plt.title('Gene Expression Patterns')
plt.legend(loc='right',bbox_to_anchor=(1.2,0.5))
plt.xlabel('Distance From Center of Wound (um)')
plt.ylabel(' Expression')

plt.show()

In [ ]:
plt.figure(figsize=(12,6),dpi=120)
for i in range(len(wounded_df)):
    std = np.std(np.array(wounded_df.iloc[i]))
    rsd = std/np.mean(np.array(wounded_df.iloc[i]))
    if rsd>1:
        plt.plot(bins,np.array(wounded_df.iloc[i])/std,label=wounded_df.index[i])
plt.rc('font', size=15) 
plt.rc('axes', titlesize=15)
plt.title('Gene Expression Patterns')
plt.legend(loc='right',bbox_to_anchor=(1.2,0.5))
plt.xlabel('Distance From Center of Wound (um)')
plt.ylabel(' Z score')

plt.show()

In [ ]:
A

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
nbins = 30
rmin = 150
rmax = 1600
bins_min = np.linspace(rmin,rmax,nbins)
A = np.array(wounded_df.iloc[2])
f = interp1d(bins,A)
plt.plot(bins,A)
plt.plot(bins_min,f(bins_min))
plt.show()

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
nbins = 30
rmin = 150
rmax = 1600
bins_min = np.linspace(rmin,rmax,nbins)
plt.figure(figsize=(12,6),dpi=120)
for i in range(len(wounded_df)):
    A = np.array(wounded_df.iloc[i])
    if np.mean(A)>0.03:
        f = interp1d(bins,A)
        X = f(bins_min)
        rsd =np.std(X)/np.mean(X)
        if rsd>1:
            plt.plot(bins_min,np.log2(X),label=wounded_df.index[i])
plt.rc('font', size=15) 
plt.rc('axes', titlesize=15)
plt.title('Gene Expression Patterns')
plt.legend(loc='right',bbox_to_anchor=(1.2,0.5))
plt.xlabel('Distance From Center of Wound (um)')
plt.ylabel(' Log2 Expression')
plt.show()

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
nbins = 30
rmin = 150
rmax = 1600
bins_min = np.linspace(rmin,rmax,nbins)
plt.figure(figsize=(12,6),dpi=120)
for i in range(len(wounded_df)):
    A = np.array(wounded_df.iloc[i])
    if np.mean(A)>0.03:
        rsd =np.std(A)/np.mean(A)
        if rsd>1:
            plt.plot(bins,np.log2(A),label=wounded_df.index[i])
plt.rc('font', size=15) 
plt.rc('axes', titlesize=15)
plt.title('Gene Expression Patterns')
plt.legend(loc='right',bbox_to_anchor=(1.2,0.5))
plt.xlabel('Distance From Center of Wound (um)')
plt.ylabel(' Log2 Expression')
plt.show()

In [ ]:
np.percentile(wounded,50)

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline
temp = spotcalls
x = temp.CoordX+2049*0.109
y = temp.CoordY
z = temp.z
plt.scatter(x,y,c=z,s=1,alpha=0.1)
plt.show()

In [ ]:
print(len(spotcalls[spotcalls.CoordX>1000]))
print(len(spotcalls[spotcalls.CoordX<1000]))

In [ ]:
print(np.mean(spotcalls[spotcalls.CoordX>1000].z))
print(np.mean(spotcalls[spotcalls.CoordX<1000].z))

In [ ]:
rsd

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
nbins = 30
rmin = 150
rmax = 1600
bins_min = np.linspace(rmin,rmax,nbins)
plt.figure(figsize=(12,6),dpi=120)
for i in range(len(log_fold_change)):
    A = np.array(wounded_df.iloc[i])
    B = np.array(log_fold_change_df.iloc[i])
    if np.mean(A)>0.03:
        rsd =np.std(A)/np.mean(A)
        if rsd>1:
            plt.plot(bins,B,label=wounded_df.index[i])
plt.rc('font', size=15) 
plt.rc('axes', titlesize=15)
plt.title('Gene Expression Patterns')
plt.legend(loc='right',bbox_to_anchor=(1.2,0.5))
plt.xlabel('Distance From Center of Wound (um)')
plt.ylabel(' Log2 Expression')
plt.show()

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
nbins = 30
rmin = 150
rmax = 1600
bins_min = np.linspace(rmin,rmax,nbins)
plt.figure(figsize=(12,6),dpi=120)
for i in range(len(log_fold_change)):
    A = np.array(wounded_df.iloc[i])
    B = np.array(log_fold_change_df.iloc[i])
    if np.mean(A)>0.03:
        f = interp1d(bins,B)
        X = f(bins_min)
        rsd =np.std(X)/np.mean(X)
        if rsd>0.3:
            plt.plot(bins_min,X,label=wounded_df.index[i])
plt.rc('font', size=15) 
plt.rc('axes', titlesize=15)
plt.title('Gene Expression Patterns')
plt.legend(loc='right',bbox_to_anchor=(1.2,0.5))
plt.xlabel('Distance From Center of Wound (um)')
plt.ylabel(' Log2 Fold Expression')
plt.show()

In [ ]:
wounded_downsampled = np.zeros((wounded.shape[0],30))
for i in range(wounded.shape[0]):
    f = interp1d(bins,B)
    wounded_downsampled[i,:] = f(bins_min)

In [ ]:
wounded_df_downsample = pd.DataFrame(data=wounded_downsampled,columns=bins_min,index=wounded_dist_hist.keys())

In [ ]:
import pandas as pd
nbins=30
log_fold_change = np.zeros((len(spotcalls.gene.unique()),nbins))
unwounded = np.zeros((len(spotcalls.gene.unique()),nbins))
wounded = np.zeros((len(spotcalls.gene.unique()),nbins))
for i,gene in enumerate(wounded_dist_hist.keys()):
    wounded_gene = wounded_dist_hist[gene]
    f = interp1d(bins,wounded_gene)
    wounded_gene = f(bins_min)
    unwounded_gene = unwounded_dist_hist[gene]
    f = interp1d(bins,unwounded_gene)
    unwounded_gene = f(bins_min)
    log = np.log2(wounded_gene/unwounded_gene)
    unwounded[i,:] = np.log2(unwounded_gene)
    wounded[i,:] = np.log2(wounded_gene)
    log_fold_change[i,:] = log

log_fold_change_df = pd.DataFrame(data=log_fold_change,columns=bins_min,index=wounded_dist_hist.keys())
wounded_df = pd.DataFrame(data=wounded,columns=bins_min,index=wounded_dist_hist.keys())
unwounded_df = pd.DataFrame(data=unwounded,columns=bins_min,index=wounded_dist_hist.keys())

In [ ]:
row

In [ ]:
df = log_fold_change_df
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
df = df.drop(columns=bins_min[-1])

In [ ]:
import seaborn as sns
sns.clustermap(df,col_cluster=False,cmap='bwr',center=0)

In [ ]:
edge_pos=[]
x_min=-2000
x_max=-1000
y_min=1250
y_max=1750

for pos in md.image_table.Position.unique():
    x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
    if (x>x_min)&(x<x_max)&(y>y_min)&(y<y_max):
        edge_pos.append(pos)
        print(pos, x,y)

In [ ]:
temp_df.columns

In [ ]:
import plotly 
import plotly.graph_objs as go
rgb_colors = []
rgb_convert = []
for i in range(171):
    rgb_convert.append("rgb({0},{1},{2})".format(np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255)))

    pos = 'Pos77'
gene = 'Tnfrsf21'
temp_df = spotcalls[(spotcalls.posname=='Pos77')|(spotcalls.posname=='Pos92')|(spotcalls.posname=='Pos62')]
temp_df = temp_df[temp_df.gene==gene]
plotly.offline.init_notebook_mode(connected=True)
X = temp_df.CoordX
Y = temp_df.CoordY
Z = temp_df.z
C = rgb_convert[temp_df.cword_idx]


plotly.offline.iplot({
    "data": [go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=2, opacity=0.5))]
})
#url=plotly.offline.iplot(fig, filename='your_filename')

In [ ]:
import plotly 
import plotly.graph_objs as go

gene = 'Nfkb1'
temp_df = spotcalls[(spotcalls.posname=='Pos77')|(spotcalls.posname=='Pos92')|(spotcalls.posname=='Pos62')]
temp_df = temp_df[temp_df.gene==gene]
plotly.offline.init_notebook_mode(connected=True)
X = temp_df.CoordX
Y = temp_df.CoordY
Z = temp_df.z


plotly.offline.iplot({
    "data": [go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=2, opacity=0.5))]
})
#url=plotly.offline.iplot(fig, filename='your_filename')

In [ ]:
import plotly 
import plotly.graph_objs as go
rgb_colors = []
rgb_convert = []
pos = 'Pos77'
gene = 'Tnfrsf21'
temp_df = spotcalls[(spotcalls.CoordX>-2500)&(spotcalls.CoordX<-1750)&(spotcalls.CoordY>1000)&(spotcalls.CoordY<2000)&(spotcalls.gene==gene)]
plotly.offline.init_notebook_mode(connected=True)
X = temp_df.CoordX
Y = temp_df.CoordY
Z = temp_df.z

plotly.offline.iplot({
    "data": [go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=2, opacity=0.1))]
})
#url=plotly.offline.iplot(fig, filename='your_filename')

In [ ]:
import plotly 
import plotly.graph_objs as go
rgb_colors = []
rgb_convert = []
pos = 'Pos77'
gene = 'Tnfrsf21'
temp_df = spotcalls[(spotcalls.CoordX<1000)&(spotcalls.gene==gene)&(spotcalls.ave>1000)]
plotly.offline.init_notebook_mode(connected=True)
X = temp_df.CoordX
Y = temp_df.CoordY
Z = temp_df.ave

plotly.offline.iplot({
    "data": [go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=2, opacity=0.1))]
})
#url=plotly.offline.iplot(fig, filename='your_filename')

In [ ]:
np.stack(temp_df.centroid)[:,0]

In [ ]:
pos = 'Pos62'
% matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=40)[:,:,0]
plt.imshow(im,cmap='Greys')
temp_df = spotcalls[spotcalls.posname==pos]
plt.scatter(np.stack(temp_df.centroid)[:,1],np.stack(temp_df.centroid)[:,0],c=temp_df.cword_idx,s=10,alpha=0.1)
plt.show()

In [ ]:
% matplotlib notebook
from sklearn.preprocessing import normalize
im = md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0]
thresh = np.percentile(im.ravel(),99.5)
im[im>thresh]=thresh
plt.imshow(im,cmap='Greys')
temp_df = spotcalls[(spotcalls.posname==pos)&(spotcalls.ave>1000)]
plt.scatter(np.stack(temp_df.centroid)[:,1],np.stack(temp_df.centroid)[:,0],c=temp_df.cword_idx,s=5)
plt.show()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib as mpl
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
from sklearn.preprocessing import normalize
from scipy import ndimage
from skimage import morphology,filters
import pywt
import numpy as np
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig = plt.figure(figsize=(3,3),dpi=4000)
ax = fig.add_subplot(111)
artists = []
wavelet = 'db10'
keep_list=[4,5,6]
scale = int(2048*0.109)
BIGGER_SIZE = 12
plt.rc('font', size=BIGGER_SIZE, family='Arial')          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        if (x<-100)&(y<2750):
            im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0])
            thresh = np.percentile(im.ravel(),99)
            im[im>thresh]=thresh
            coeffs = pywt.wavedec2(im,wavelet)
            for i in range(0,len(coeffs)+1):
                if i in keep_list:
                    continue
                coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
            Result = pywt.waverec2(coeffs,wavelet)
            thresh = np.percentile(Result.ravel(),99.9)
            Result[Result>thresh]=thresh
            Result[im<2**8.5]=np.nan
            if (x>-2500)&(x<-500)&(y>800)&(y<2000):
                im_bin = im>500
            else:
                im_bin = im>2000
            im_bin = ndimage.binary_dilation(im_bin,iterations=5)
            im_bin = morphology.remove_small_holes(im_bin, 2000)
            im_bin = morphology.remove_small_objects(im_bin, 10000)
            im_bin = morphology.erosion(im_bin)
            Result[im_bin==False]=np.nan
            Result = Result-np.nanpercentile(Result.ravel(),1)
            Result = Result/np.nanpercentile(Result.ravel(),99)
            bb = Bbox.from_bounds(x,y,scale,scale)
            bb2 = TransformedBbox(bb,ax.transData)
            bbox_image = BboxImage(bb2,
                                norm = norm,
                                origin=None,
                                clip_on=False,
                                cmap='Greys')
            bbox_image.set_data(Result)
            ax.add_artist(bbox_image)
ax.set_xlim(-3500,100)
ax.set_ylim(0,3000)
plt.xlabel('X Stage Coordinate (um)')
plt.ylabel('Y Stage Coordinate (um)')
plt.suptitle('Wounded Cornea')
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea')

In [ ]:
import matplotlib as mpl
norm = mpl.colors.Normalize(vmin=0,vmax=1)
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=BIGGER_SIZE, family='Arial')          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.figure(figsize=(8,8))
plt.imshow(Result,cmap='Greys',norm=norm)
plt.xlabel('X Stage Coordinate (um)')
plt.ylabel('Y Stage Coordinate (um)')
plt.suptitle('Wounded Cornea')
plt.show()

In [ ]:
%matplotlib inline
plt.figure(figsize=(8,8))
plt.imshow(Result,cmap='Greys')

In [ ]:
% matplotlib inline
import pywt
from sklearn.preprocessing import normalize
wavelet = 'db10'
keep_list=[4,5,6]
pos ='Pos62'
im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0])
thresh = np.percentile(im.ravel(),99)
im[im>thresh]=thresh
coeffs = pywt.wavedec2(im,wavelet)
for i in range(0,len(coeffs)+1):
    if i in keep_list:
        continue
    coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
Result = pywt.waverec2(coeffs,wavelet)
thresh = np.percentile(Result.ravel(),99.9)
Result[Result>thresh]=thresh
Result[im<2**8.5]=np.nan
Result = Result-np.nanmin(Result)
im_bin = im>700
im_bin = ndimage.binary_dilation(im_bin,iterations=5)
im_bin = morphology.remove_small_holes(im_bin, 2000)
im_bin = morphology.remove_small_objects(im_bin, 10000)
im_bin = morphology.erosion(im_bin)
Result[im_bin==False]=np.nan
Result = Result-np.nanpercentile(Result.ravel(),1)
Result = Result/np.nanpercentile(Result.ravel(),99)
plt.imshow(Result,cmap='Greys')
plt.show()

In [ ]:
% matplotlib inline
import pywt
from sklearn.preprocessing import normalize
wavelet = 'db10'
keep_list=[4,5,6]
pos ='Pos7'
im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0])
thresh = np.percentile(im.ravel(),99)
im[im>thresh]=thresh
coeffs = pywt.wavedec2(im,wavelet)
for i in range(0,len(coeffs)+1):
    if i in keep_list:
        continue
    coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
Result = pywt.waverec2(coeffs,wavelet)
thresh = np.percentile(Result.ravel(),99.9)
Result[Result>thresh]=thresh
Result[im<2**8.5]=np.nan
Result = Result-np.nanmin(Result)
im_bin = im>700
im_bin = ndimage.binary_dilation(im_bin,iterations=5)
im_bin = morphology.remove_small_holes(im_bin, 2000)
im_bin = morphology.remove_small_objects(im_bin, 10000)
im_bin = morphology.erosion(im_bin)
Result[im_bin==False]=np.nan
Result = Result-np.nanpercentile(Result.ravel(),1)
Result = Result/np.nanpercentile(Result.ravel(),99)
plt.imshow(Result,cmap='Greys')
plt.show()

In [ ]:
np.nanpercentile(Result.ravel(),1)

In [ ]:
% matplotlib inline
import pywt
from sklearn.preprocessing import normalize
wavelet = 'db10'
keep_list=[4,5,6]
pos ='Pos25'
im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0])
thresh = np.percentile(im.ravel(),99)
im[im>thresh]=thresh
coeffs = pywt.wavedec2(im,wavelet)
for i in range(0,len(coeffs)+1):
    if i in keep_list:
        continue
    coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
Result = pywt.waverec2(coeffs,wavelet)
thresh = np.percentile(Result.ravel(),99.9)
Result[Result>thresh]=thresh
Result[im<2**8.5]=np.nan
Result = Result-np.nanmin(Result)
im_bin = im>2000
im_bin = ndimage.binary_dilation(im_bin,iterations=5)
im_bin = morphology.remove_small_holes(im_bin, 2000)
im_bin = morphology.remove_small_objects(im_bin, 10000)
im_bin = morphology.erosion(im_bin)
Result[im_bin==False]=np.nan
Result = Result/np.nanmean(Result)
plt.imshow(Result,cmap='Greys')
plt.show()

In [ ]:
% matplotlib inline
for pos in md.image_table.Position.unique():
    im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0])
    plt.imshow(im,cmap='Greys')
    plt.title(pos)
    plt.show()

In [ ]:
plt.hist(img.ravel())

In [ ]:
% matplotlib inline
from scipy import ndimage
from skimage import morphology,filters
temp = Result_Pos62.copy()
img = im.copy()
im_bin = img>500
im_bin = ndimage.binary_dilation(im_bin,iterations=5)
im_bin = morphology.remove_small_holes(im_bin, 2000)
im_bin = morphology.remove_small_objects(im_bin, 10000)
im_bin = morphology.erosion(im_bin)
temp[im_bin==False]=0
plt.imshow(temp,cmap='Greys')
plt.show()

In [ ]:
plt.hist(Result.ravel())

In [ ]:
% matplotlib inline
import pywt
from sklearn.preprocessing import normalize
wavelet = 'db10'
keep_list=[4,5,6]
pos ='Pos7'
im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0])
thresh = np.percentile(im.ravel(),99)
im[im>thresh]=thresh
coeffs = pywt.wavedec2(im,wavelet)
for i in range(0,len(coeffs)+1):
    if i in keep_list:
        continue
    coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
Result = pywt.waverec2(coeffs,wavelet)
thresh = np.percentile(Result.ravel(),99.9)
Result[Result>thresh]=thresh
Result = Result-Result.min()
Result = normalize(Result)
Result[im<2**8.5]=np.nan
plt.imshow(Result,cmap='Greys')
plt.show()

In [ ]:
np.nanmin(Result)

In [ ]:
np.nanmin(Result_Pos62)

In [ ]:
%matplotlib notebook
plt.imshow(im,cmap='Greys')

In [ ]:
%matplotlib notebook
plt.hist(np.log2(Result.ravel()),bins=1000)
plt.show()

In [ ]:
2**8